# 아영 - Story Generator

# 1. Ready

## 1.1 기본 패키지 임포트

In [1]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel


## 1.2 학습 모델 다운로드

In [2]:
MODEL_NAME = "skt/kogpt2-base-v2"

### 토크나이저 load

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)

TOKENS_DICT = {
    'bos_token':'<s>',
    'eos_token':'</s>',
    'unk_token':'<unk>',
    'pad_token':'<pad>',
    'mask_token':'<mask>'
}

# 특수 토큰이 토크나이저에 추가되고 모델은 수정된 토크나이저에 맞게 임베딩의 크기를 조정
tokenizer.add_special_tokens(TOKENS_DICT)

print(tokenizer.special_tokens_map)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


### 미세조정학습 모델 load

In [4]:
from util.generator import sample_sequence as gen

In [5]:
from util.model import model_loading as load

In [39]:
checkpointPath = "./modelCheckpoint/KoGPT2_checkpoint_novelstart.tar"
loading = True



"""
talestart_from1980_newdata 은호
KoGPT2_checkpoint_novelstart 지현
tale-novel-batch-3-1 창균
talestart680w 현지
travel2 아영
"""

'\ntalestart_from1980_newdata 은호\nKoGPT2_checkpoint_novelstart 지현\ntale-novel-batch-3-1 창균\ntalestart680w 현지\ntravel2 아영\n'

In [40]:
model,ckp = load(checkpointPath, PU = 'cpu', status = loading)

In [41]:
print(ckp['epoch'], ckp['loss'])

620 tensor(0.0886, requires_grad=True)


In [42]:
vocab_path = "./data/정제/index_tale_plus_novel.txt"

In [43]:
p1, p2, p3 = 1,2,1

max_sent = [p1,p2//2, p2//2 ,p3]

context = "어제"


In [45]:
from util.generator import sample_sequence
from util.similarity import similarity

In [71]:
# 1번 문단 생성
print("1번째 문단")
result = []
p = sample_sequence(
    vocab=vocab_path,
    model=model, 
    length = max_sent[0]*30,
    context=context,
    num_samples=1, 
    repetition_penalty=2.0,
    top_p=0.9,
    tokenizer = tokenizer)
p = tokenizer.decode(p[0,:].tolist())
result.append(p)

# 2~4번 문단 생성
for i in range(len(max_sent[1:])):
    print(f"{i+2}번째 문단")
    sim = 0
    while(sim < 0.02):
        next_p = sample_sequence(
            vocab=vocab_path,
            model=model, 
            length=max_sent[i+1]*30,
            context=p, 
            num_samples=1, 
            repetition_penalty=2.0,
            top_p=0.9,
            tokenizer = tokenizer)
        next_p = tokenizer.decode(next_p[0,:].tolist())
        sim = similarity(p, next_p)
        print(sim)

    result.append(next_p)
    p = next_p

  7%|▋         | 2/30 [00:00<00:01, 14.20it/s]

1번째 문단


31it [00:03,  8.52it/s]
  3%|▎         | 1/30 [00:00<00:04,  5.93it/s]

Word not in corpus => 0 skipped
2번째 문단


31it [00:06,  4.75it/s]
  0%|          | 0/30 [00:00<?, ?it/s]

Word not in corpus => 0 skipped
0.5404512628201446
3번째 문단


31it [00:10,  2.99it/s]
  0%|          | 0/30 [00:00<?, ?it/s]

Word not in corpus => 0 skipped
0.7697396770788578
4번째 문단


31it [00:13,  2.26it/s]

Word not in corpus => 0 skipped
0.7952821598786765


In [73]:
result

['어제 \n굿이 시작된 모양이었다.\n마당에 흩어져 있던 사람들이 좁은 마루로 몰려 가고 있었다.\n경칩이 지났다고는 하지만 바닷가여서인지 유난히 밤바람',
 '어제 \n굿이 시작된 모양이었다.\n마당에 흩어져 있던 사람들이 좁은 마루로 몰려 가고 있었다.\n경칩이 지났다고는 하지만 바닷가여서인지 유난히 밤바람이 차가운 곳이었다.\n나는 마당 구석으로 가서 휴대폰으로 집에 전화를 걸었다.\n무녀의 에에루하는 불분명한 소리가 들려왔다.\n어쩌면 엄마가 남편에게',
 '어제 \n굿이 시작된 모양이었다.\n마당에 흩어져 있던 사람들이 좁은 마루로 몰려 가고 있었다.\n경칩이 지났다고는 하지만 바닷가여서인지 유난히 밤바람이 차가운 곳이었다.\n나는 마당 구석으로 가서 휴대폰으로 집에 전화를 걸었다.\n무녀의 에에루하는 불분명한 소리가 들려왔다.\n어쩌면 엄마가 남편에게 전화를 할 생각이었으나 나는 달만 쳐다보며 계속 시간을 미루고 있었다.\n남편은 전화를 받지 않았다.\n강?수 데리고 8졌어요.\n예전화를 준비',
 '어제 \n굿이 시작된 모양이었다.\n마당에 흩어져 있던 사람들이 좁은 마루로 몰려 가고 있었다.\n경칩이 지났다고는 하지만 바닷가여서인지 유난히 밤바람이 차가운 곳이었다.\n나는 마당 구석으로 가서 휴대폰으로 집에 전화를 걸었다.\n무녀의 에에루하는 불분명한 소리가 들려왔다.\n어쩌면 엄마가 남편에게 전화를 할 생각이었으나 나는 달만 쳐다보며 계속 시간을 미루고 있었다.\n남편은 전화를 받지 않았다.\n강?수 데리고 8졌어요.\n예전화를 준비해봐야 했다.\n회사에 얼굴이 젖 위해서는벌서 가슴에 엄마이라면 천장을 향해 가기 필요하지는 못했다.\n볼까. 앞으로도 벌써 조금도 스스로처럼 애 산']